In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np
import pandas as pd

from scipy.optimize import curve_fit

import gm2
import trfp
import plotting_functions as plt2
import analysis_helper as helper
import helper_function_candidates as helper_old

Welcome to JupyROOT 6.22/02


In [4]:
def __multipole(order, skew, strength, x_pos, y_pos):
    """Returns the magnitude of a B-field given multipole parameters
    and position.
    """
    # Takes a multipole strength A normalized to 4.5 cm
    r_pos = np.sqrt(x_pos**2 + y_pos**2)
    theta = np.arctan2(y_pos, x_pos)
    if skew == 0:
        b_magnitude = strength * (r_pos/4.5)**order * np.cos(order*theta)
    if skew == 1:
        b_magnitude = strength * (r_pos/4.5)**order * np.sin(order*theta)
    return b_magnitude

In [5]:
_MULTIPOLE_ORDER = [0, 1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 7, 7]
_MULTIPOLE_SKEW = [0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1]
_MULTS = np.array([__multipole(_MULTIPOLE_ORDER[i], _MULTIPOLE_SKEW[i], 1, trfp.TR_X, trfp.TR_Y) for i in range(14)])
_MULTS[np.abs(_MULTS) < 1.0e-9] = 0 

THETA_TR = np.linalg.pinv(np.transpose(_MULTS))
THETA_TR = np.insert(THETA_TR, 12, np.zeros(17), 0)
THETA_TR = np.append(THETA_TR, np.zeros([2,17]), 0)
THETA_TR[np.abs(THETA_TR) < 1.0e-9] = 0

In [7]:
print THETA_TR.shape

(17, 17)


In [12]:
_MULTIPOLE_ORDER = [0, 1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 7, 7, 8, 10, 12]
_MULTIPOLE_SKEW = [0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0]
_MULTS = np.array([__multipole(_MULTIPOLE_ORDER[i], _MULTIPOLE_SKEW[i], 1, trfp.TR_X, trfp.TR_Y) for i in range(17)])
_MULTS[np.abs(_MULTS) < 1.0e-9] = 0 

THETA_TR_2 = np.linalg.inv(np.transpose(_MULTS))
THETA_TR_2[np.abs(THETA_TR_2) < 1.0e-9] = 0

In [15]:
print THETA_TR_2

[[ 1.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  1.07142857e-01  1.85576872e-01
   2.14285714e-01  1.85576872e-01  1.07142857e-01  0.00000000e+00
  -1.07142857e-01 -1.85576872e-01 -2.14285714e-01 -1.85576872e-01
  -1.07142857e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00 -2.14285714e-01 -1.85576872e-01 -1.07142857e-01
   0.00000000e+00  1.07142857e-01  1.85576872e-01  2.14285714e-01
   1.85576872e-01  1.07142857e-01  0.00000000e+00 -1.07142857e-01
  -1.85576872e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00 -2.38598836e-01 -2.38598836e-01
   0.00000000e+00  

In [ ]:
def _choose_theta(st):
    if trfp.STATION_PROBE_NUMBER[st] == 4:
        if st == 41:
            theta_fp = trfp.THETA_FP_4_ST41
        elif (st == 37) | (st == 39):
            theta_fp = trfp.THETA_FP_4_ST37_ST39
        else:
            theta_fp = trfp.THETA_FP_4
    elif trfp.STATION_PROBE_NUMBER[st] == 6:
        theta_fp = trfp.THETA_FP_6
    else:
        raise Exception('Invalid number of station probes.')
    return theta_fp


def calc_moment_df(interp_df):
    
    tr_run = 'tr_phi' in interp_df.columns
    
    moment_df = pd.DataFrame(index=interp_df.index)
    
    # calculate trolley moments if needed
    if tr_run:
        moment_df['tr_phi'] = interp_df['tr_phi'].copy()
        print 'Calculating trolley moments.',
        theta_tr = THETA_TR
        for m in np.arange(17):
            tr_probes = ['tr'+str(probe) for probe in np.arange(17)]
            moment_df['tr,m'+str(m+1)] = interp_df[tr_probes].dot(theta_tr[m])

    # create the 72*6 fixed probe moments
    for station in np.arange(72):
        print '\rCalculating station ' + str(station) + ' moments.',
        fp_st = ['fp'+str(fp) for fp in trfp.STATION_PROBE_ID[station]]

        # choose proper theta matrix
        theta_fp = _choose_theta(station)

        # step through m values
        for m in np.arange(len(trfp.STATION_PROBE_ID[station])):
            stm = 'st'+str(station)+',m'+str(m+1)
            moment_df[stm] = interp_df[fp_st].dot(theta_fp[m])
        if len(trfp.STATION_PROBE_ID[station]) == 4:
            moment_df['st'+str(station)+',m5'] = np.nan
            moment_df['st'+str(station)+',m6'] = np.nan

    # Interpolate the 6-probe m5 and m6 into the 4-probe m5 and m6        
    for st in range(72):
        if trfp.STATION_PROBE_NUMBER[st] == 4:
            wt = trfp.STATION_BARCODE_PHI[(st+1)%72] - trfp.STATION_BARCODE_PHI[(st-1)%72]
            w1 = trfp.STATION_BARCODE_PHI[(st+1)%72] - trfp.STATION_BARCODE_PHI[st]
            w2 = trfp.STATION_BARCODE_PHI[st] - trfp.STATION_BARCODE_PHI[(st-1)%72]

            for m in [5, 6]:
                stm = 'st'+str(st)+',m'+str(m)
                if not np.isnan(moment_df[stm].iloc[0]):
                    print stm + ' is not nan.'
                    continue
                stm1 = 'st'+str((st-1)%72)+',m'+str(m)
                stm2 = 'st'+str((st+1)%72)+',m'+str(m)

                moment_df[stm] = (w1*moment_df[stm1] + w2*moment_df[stm2])/wt

    print '\rFinished calculating all moments for ' + str(moment_df.shape[0]) + ' events.'
    
    return moment_df